In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install pickle5

     |████████████████████████████████| 256 kB 8.4 MB/s 


In [11]:
import pickle5 as pickle

In [12]:
data_dir = 'drive/MyDrive/stat940/dc2'

with open(f"{data_dir}/train.pickle", "rb") as f:
    train_dict = pickle.load(f)

with open(f"{data_dir}/test.pickle", "rb") as f:  
    test_dict = pickle.load(f)

In [17]:
!pip install folium==0.2.1
!pip install tensorboardX
!pip install transformers==2.3.0

     |████████████████████████████████| 69 kB 3.9 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=5ff7dfc87422bad8fccd227e6bb83239611e1bfc3d3d8d7eb6bf519c920221f1
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [18]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Finetuning the library models for sequence classification on GLUE (Bert, XLM, XLNet, RoBERTa)."""

from __future__ import absolute_import, division, print_function

import argparse
import glob
import logging
import os
import random
import csv
import codecs
import functools

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset, Dataset)
from torch.utils.data.distributed import DistributedSampler
from tensorboardX import SummaryWriter
from tqdm import tqdm, trange

from transformers import (WEIGHTS_NAME, BertConfig,
                    BertForSequenceClassification, BertTokenizer)

from transformers import AdamW, get_linear_schedule_with_warmup

from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers import DataProcessor, InputExample, InputFeatures

logger = logging.getLogger(__name__)

In [28]:
#ALL_MODELS = sum((tuple(
#    conf.pretrained_config_archive_map.keys()) for conf in (
#        BertConfig)), ())

MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer)
}


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


class MyTestDataset(Dataset):
    """Dataset during test time"""

    def __init__(self, tensor_data, sents):
        assert len(tensor_data) == len(sents)
        self.tensor_data = tensor_data
        self.rows = sents
        
    def __len__(self):
        return len(self.tensor_data)

    def __getitem__(self, idx):
        return (self.tensor_data[idx], self.rows[idx])
    
def evaluate_test(args, model, tokenizer, prefix=""):
    
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    processor = PairProcessor()
    output_mode = "classification"
    
    cached_features_file = os.path.join(args.data_dir, 'cached_{}_{}_{}_{}'.format(
        'test',
        'bert',
        str(args.max_seq_length),
        'pair_order'))
    
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
        lines = torch.load(cached_features_file + '_lines')
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()

        examples, lines = processor.get_test_examples(args.data_dir)
        features = convert_examples_to_features(
            examples,
            tokenizer,
            label_list=label_list,
            max_length=args.max_seq_length,
            output_mode=output_mode,
            pad_on_left=False,                 # pad on the left for xlnet
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
        )
        if args.local_rank in [-1, 0]:
            logger.info(
                "Saving features into cached file %s", 
                cached_features_file)
            torch.save(features, cached_features_file)
            torch.save(lines, cached_features_file + '_lines')
        
    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier() 

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(
        [f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(
        all_input_ids, 
        all_attention_mask, 
        all_token_type_ids, 
        all_labels
        )

    
    eval_outputs_dirs = (args.output_dir,)
    file_h = codecs.open(args.data_dir + "test_results.tsv", "w", "utf-8")
    outF = csv.writer(file_h, delimiter='\t')

    results = {}
    for eval_output_dir in eval_outputs_dirs:
        eval_dataset = MyTestDataset(dataset, lines)
        
        if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        eval_sampler = SequentialSampler(eval_dataset)
        
        eval_dataloader = DataLoader(
            eval_dataset, 
            sampler=eval_sampler, 
            batch_size=args.eval_batch_size
            )

        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            row = batch[1]
            rows = {
                'guid': row[0],
                'text_a': row[1],
                'text_b': row[2],
                'labels': row[3],
                'pos_a': row[4],
                'pos_b':row[5]
            }
            del row
            batch = tuple(t.to(args.device) for t in batch[0])

            with torch.no_grad():
                inputs = {
                        'input_ids':      batch[0],
                        'attention_mask': batch[1],
                        'token_type_ids': batch[2],
                        'labels':         batch[3]}

                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]
                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            logits = logits.detach().cpu().numpy()
            tmp_pred = np.argmax(logits, axis=1)
            for widx in range(logits.shape[0]):
                outF.writerow([rows['guid'][widx], rows['text_a'][widx], \
                rows['text_b'][widx], rows['labels'][widx], \
                rows['pos_a'][widx], rows['pos_b'][widx], \
                logits[widx][0], logits[widx][1], tmp_pred[widx]])
            if preds is None:
                preds = logits
                out_label_ids = inputs['labels'].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits, axis=0)
                out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        preds = np.argmax(preds, axis=1)

        result = compute_metrics("mnli", preds, out_label_ids)
        results.update(result)

        file_h.close()
        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    return results

def evaluate(args, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    
    eval_outputs_dirs = (args.output_dir,)

    results = {}
    for eval_output_dir in eval_outputs_dirs:
        eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

        if not os.path.exists(eval_output_dir) and args.local_rank in [-1, 0]:
            os.makedirs(eval_output_dir)

        args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        # Eval!
        logger.info("***** Running evaluation {} *****".format(prefix))
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None
        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            model.eval()
            batch = tuple(t.to(args.device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'token_type_ids': batch[2],
                          'labels':         batch[3]
                        }
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs['labels'].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        preds = np.argmax(preds, axis=1)

        result = compute_metrics("mnli", preds, out_label_ids)
        results.update(result)

        output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results {} *****".format(prefix))
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    return results


def load_and_cache_examples(args, tokenizer, evaluate=False):
    if args.local_rank not in [-1, 0] and not evaluate:
        torch.distributed.barrier()  

    processor = PairProcessor()
    output_mode = 'classification'
    
    cached_features_file = os.path.join(
        args.data_dir, 'cached_{}_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        'bert',
        str(args.max_seq_length),
        'pair_order'))

    if os.path.exists(cached_features_file):
        logger.info(
            "Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info(
            "Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args.data_dir) if evaluate else processor.get_train_examples(args.data_dir)
        features = convert_examples_to_features(examples,
                                    tokenizer,
                                    label_list=label_list,
                                    max_length=args.max_seq_length,
                                    output_mode=output_mode,
                                    pad_on_left=False,                 # pad on the left for xlnet
                                    pad_token=tokenizer.convert_tokens_to_ids(
                                        [tokenizer.pad_token])[0],
                                    pad_token_segment_id=0,
                                    )
        if args.local_rank in [-1, 0]:
            logger.info(
                "Saving features into cached file %s", cached_features_file)
            torch.save(features, cached_features_file)
        
    if args.local_rank == 0 and not evaluate:
        torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(
        [f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features], dtype=torch.long)

    all_labels = torch.tensor(
        [f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(
        all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset

class PairProcessor(DataProcessor):
    """Pair Processor for the pair ordering task."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")
    
    def get_test_examples(self, data_dir):
        return self._create_test_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")
    
    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (_, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, line[0])
            try:
                text_a = line[1].lower()
                text_b = line[2].lower()
                label = line[3]
            except IndexError:
                print('cannot read the line: ' + line)
                continue
            examples.append(InputExample(
                                        guid=guid, 
                                        text_a=text_a, 
                                        text_b=text_b, 
                                        label=label
                                    ))
        return examples
    
    def _create_test_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples, rows = [], []
        for (_, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, line[0])
            try:
                text_a = line[1].lower()
                text_b = line[2].lower()
                label = line[3]
            except IndexError:
                print('cannot read the line: ' + line)
                continue
            examples.append(InputExample(
                                    guid=guid, 
                                    text_a=text_a, 
                                    text_b=text_b, 
                                    label=label
                            ))
            rows.append(line)
        return examples, rows

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)

# Set seed
set_seed(42)

_, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)

tokenizer = tokenizer_class.from_pretrained('bert-base-uncased')
model = model_class.from_pretrained('bert-base-uncased')

model.to(device)

03/04/2022 19:23:24 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/04/2022 19:23:25 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
03/04/2022 19:23:25 - INFO - transformers.configuration_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1":

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element